In [11]:
import neo4j

host = "bolt://localhost"
username = "neo4j"
password = "1234qwer"

driver = neo4j.GraphDatabase.driver(f'{host}', auth=(username, password))

session = driver.session()

In [4]:
import psycopg2

pg_host = "localhost"
pg_database = "graph"
pg_user = "postgres"
pg_password = "postgres"

# Establish PostgreSQL connection
pg_conn = psycopg2.connect(
    host=pg_host,
    database=pg_database,
    user=pg_user,
    password=pg_password
)
pg_cursor = pg_conn.cursor()

In [16]:
session.run("""
MATCH (p:Requirement)
RETURN gds.graph.project(
  'bertRequirementGraph',
  p,
  null,
  {
    sourceNodeProperties: p { .bert_embedding },
    targetNodeProperties: {}
  }
)
""")

In [18]:
records, summary, key = driver.execute_query("""
CALL gds.knn.stream('bertRequirementGraph', {
    topK: 2,
    nodeProperties: {bert_embedding: 'EUCLIDEAN'},
    randomSeed: 1337,
    concurrency: 1,
    sampleRate: 1.0,
    deltaThreshold: 0.0
})
YIELD node1, node2, similarity
RETURN gds.util.asNode(node1).id AS id1, gds.util.asNode(node2).id AS id2, similarity
ORDER BY similarity DESC
""")

for row in records:
    insert_query = """
        INSERT INTO sentence_matching (source_id, target_id, similarity, method, embedding) 
        VALUES (%s, %s, %s, %s, %s)
    """
    pg_cursor.execute(insert_query, (row['id1'], row['id2'], row['similarity'], 'EUCLIDEAN', 'BERT'))
    pg_conn.commit()